In [234]:
! pip install SALib
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from SALib.sample import saltelli
from SALib.analyze import sobol

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [235]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Read the dataset

In [236]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Agrimonia/Parte2/Recap previous thesis/Agrimonia_Dataset_v_2_0_1.csv")
pd.set_option('display.max_columns', None)
df['Latitude'] = df['Latitude'].round(2)
df['Longitude'] = df['Longitude'].round(2)

df['AQ_nh3'] = df['AQ_nh3'].replace('NaN', np.nan)
df.head()

<ipython-input-236-59baaa92da6f>:1: DtypeWarning: Columns (0,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Agrimonia/Parte2/Recap previous thesis/Agrimonia_Dataset_v_2_0_1.csv")


,IDStations,Latitude,Longitude,Time,Altitude,AQ_pm10,AQ_pm25,AQ_co,AQ_nh3,AQ_nox,AQ_no2,AQ_so2,WE_temp_2m,WE_wind_speed_10m_mean,WE_wind_speed_10m_max,WE_mode_wind_direction_10m,WE_tot_precipitation,WE_precipitation_t,WE_surface_pressure,WE_solar_radiation,WE_rh_min,WE_rh_mean,WE_rh_max,WE_wind_speed_100m_mean,WE_wind_speed_100m_max,WE_mode_wind_direction_100m,WE_blh_layer_max,WE_blh_layer_min,EM_nh3_livestock_mm,EM_nh3_agr_soils,EM_nh3_agr_waste_burn,EM_nh3_sum,EM_nox_traffic,EM_nox_sum,EM_so2_sum,LI_pigs,LI_bovine,LA_hvi,LA_lvi,LA_land_use,LA_soil_use
0,1264,46.17,9.88,2016-01-01,290,6.200e+01,5.300e+01,NaN,NaN,8.263e+01,3.772e+01,NaN,-2.212,0.5970,0.9883,S,0.00037,0,83720.0,5664000.0,88.70,93.66,99.19,1.0730,1.977,S,340.6,10.63,2.015e-01,1.462e-01,2.019e-03,6.166e-01,8.359e-01,1.720e+00,3.906e-01,0.3832,5.892,3.998,1.234,112,17.0
1,1264,46.17,9.88,2016-01-02,290,7.300e+01,6.300e+01,NaN,NaN,1.017e+02,3.884e+01,NaN,-3.063,0.4402,0.7365,SE,0.07521,5,82990.0,1335000.0,94.07,97.60,99.24,1.0530,2.321,S,238.7,21.55,2.020e-01,1.589e-01,1.956e-03,6.298e-01,8.372e-01,1.720e+00,3.911e-01,0.3832,5.892,3.997,1.234,112,17.0
2,1264,46.17,9.88,2016-01-03,290,4.400e+01,3.900e+01,NaN,NaN,6.626e+01,3.234e+01,NaN,-2.768,0.6954,1.0100,S,0.02744,5,82320.0,3728000.0,90.15,95.73,98.55,1.9810,2.942,S,407.0,22.80,2.024e-01,1.708e-01,1.890e-03,6.424e-01,8.384e-01,1.720e+00,3.916e-01,0.3832,5.892,3.997,1.234,112,17.0
3,1264,46.17,9.88,2016-01-04,290,3.100e+01,2.900e+01,NaN,NaN,6.331e+01,3.167e+01,NaN,-4.352,0.5812,0.9421,W,0.08564,5,81410.0,1846000.0,90.84,96.01,99.43,1.2080,2.284,N,536.7,12.21,2.027e-01,1.821e-01,1.822e-03,6.545e-01,8.396e-01,1.720e+00,3.923e-01,0.3832,5.892,3.997,1.234,112,17.0
4,1264,46.17,9.88,2016-01-05,290,2.700e+01,2.600e+01,NaN,NaN,7.553e+01,3.333e+01,NaN,-6.184,0.5906,0.9847,SE,0.01802,5,81530.0,1733000.0,89.31,93.53,99.05,0.8226,1.672,SE,472.4,10.15,2.031e-01,1.929e-01,1.750e-03,6.660e-01,8.409e-01,1.720e+00,3.931e-01,0.3832,5.892,3.997,1.233,112,17.0


In [237]:
df['AQ_nh3'] = df['AQ_nh3'].astype(float)
df['AQ_pm10'] = df['AQ_pm10'].astype(float)
df['AQ_pm25'] = df['AQ_pm25'].astype(float)
df['AQ_co'] = df['AQ_co'].astype(float)
df['AQ_nox'] = df['AQ_nox'].astype(float)
df['AQ_no2'] = df['AQ_no2'].astype(float)
df['AQ_so2'] = df['AQ_so2'].astype(float)
df['EM_nh3_livestock_mm'] = df['EM_nh3_livestock_mm'].astype(float)
df['EM_nh3_agr_soils'] = df['EM_nh3_agr_soils'].astype(float)
df['EM_nh3_agr_waste_burn'] = df['EM_nh3_agr_waste_burn'].astype(float)
df['EM_nh3_sum'] = df['EM_nh3_sum'].astype(float)
df['EM_nox_traffic'] = df['EM_nox_traffic'].astype(float)
df['EM_so2_sum'] = df['EM_so2_sum'].astype(float)
df['LI_pigs'] = df['LI_pigs'].astype(float)
df['LI_bovine'] = df['LI_bovine'].astype(float)
df['EM_nox_sum'] = df['EM_nox_sum'].astype(float)

In [238]:
df.head()

,IDStations,Latitude,Longitude,Time,Altitude,AQ_pm10,AQ_pm25,AQ_co,AQ_nh3,AQ_nox,AQ_no2,AQ_so2,WE_temp_2m,WE_wind_speed_10m_mean,WE_wind_speed_10m_max,WE_mode_wind_direction_10m,WE_tot_precipitation,WE_precipitation_t,WE_surface_pressure,WE_solar_radiation,WE_rh_min,WE_rh_mean,WE_rh_max,WE_wind_speed_100m_mean,WE_wind_speed_100m_max,WE_mode_wind_direction_100m,WE_blh_layer_max,WE_blh_layer_min,EM_nh3_livestock_mm,EM_nh3_agr_soils,EM_nh3_agr_waste_burn,EM_nh3_sum,EM_nox_traffic,EM_nox_sum,EM_so2_sum,LI_pigs,LI_bovine,LA_hvi,LA_lvi,LA_land_use,LA_soil_use
0,1264,46.17,9.88,2016-01-01,290,62.0,53.0,NaN,NaN,82.63,37.72,NaN,-2.212,0.5970,0.9883,S,0.00037,0,83720.0,5664000.0,88.70,93.66,99.19,1.0730,1.977,S,340.6,10.63,0.2015,0.1462,0.002019,0.6166,0.8359,1.72,0.3906,0.3832,5.892,3.998,1.234,112,17.0
1,1264,46.17,9.88,2016-01-02,290,73.0,63.0,NaN,NaN,101.70,38.84,NaN,-3.063,0.4402,0.7365,SE,0.07521,5,82990.0,1335000.0,94.07,97.60,99.24,1.0530,2.321,S,238.7,21.55,0.2020,0.1589,0.001956,0.6298,0.8372,1.72,0.3911,0.3832,5.892,3.997,1.234,112,17.0
2,1264,46.17,9.88,2016-01-03,290,44.0,39.0,NaN,NaN,66.26,32.34,NaN,-2.768,0.6954,1.0100,S,0.02744,5,82320.0,3728000.0,90.15,95.73,98.55,1.9810,2.942,S,407.0,22.80,0.2024,0.1708,0.001890,0.6424,0.8384,1.72,0.3916,0.3832,5.892,3.997,1.234,112,17.0
3,1264,46.17,9.88,2016-01-04,290,31.0,29.0,NaN,NaN,63.31,31.67,NaN,-4.352,0.5812,0.9421,W,0.08564,5,81410.0,1846000.0,90.84,96.01,99.43,1.2080,2.284,N,536.7,12.21,0.2027,0.1821,0.001822,0.6545,0.8396,1.72,0.3923,0.3832,5.892,3.997,1.234,112,17.0
4,1264,46.17,9.88,2016-01-05,290,27.0,26.0,NaN,NaN,75.53,33.33,NaN,-6.184,0.5906,0.9847,SE,0.01802,5,81530.0,1733000.0,89.31,93.53,99.05,0.8226,1.672,SE,472.4,10.15,0.2031,0.1929,0.001750,0.6660,0.8409,1.72,0.3931,0.3832,5.892,3.997,1.233,112,17.0


#Select one station

In [239]:
mydf = df[df['AQ_nh3'].notna()& df['AQ_pm10'].notna()]
mydf = mydf[mydf['IDStations']==1266]
print(mydf.shape)
mydf.head()

(1851, 41)


,IDStations,Latitude,Longitude,Time,Altitude,AQ_pm10,AQ_pm25,AQ_co,AQ_nh3,AQ_nox,AQ_no2,AQ_so2,WE_temp_2m,WE_wind_speed_10m_mean,WE_wind_speed_10m_max,WE_mode_wind_direction_10m,WE_tot_precipitation,WE_precipitation_t,WE_surface_pressure,WE_solar_radiation,WE_rh_min,WE_rh_mean,WE_rh_max,WE_wind_speed_100m_mean,WE_wind_speed_100m_max,WE_mode_wind_direction_100m,WE_blh_layer_max,WE_blh_layer_min,EM_nh3_livestock_mm,EM_nh3_agr_soils,EM_nh3_agr_waste_burn,EM_nh3_sum,EM_nox_traffic,EM_nox_sum,EM_so2_sum,LI_pigs,LI_bovine,LA_hvi,LA_lvi,LA_land_use,LA_soil_use
4384,1266,45.23,9.67,2016-01-01,65,52.0,NaN,NaN,32.2,36.21,23.50,NaN,2.1610,0.9449,1.818,E,0.000009,0,101900.0,5330000.0,92.17,97.16,99.67,1.429,2.622,NE,351.7,13.88,8.907,0.3187,0.1439,9.749,1.922,3.844,0.9099,513.5,255.2,1.548,2.084,112,17.0
4385,1266,45.23,9.67,2016-01-02,65,39.0,NaN,NaN,14.6,29.72,24.75,NaN,0.9301,1.4660,2.335,E,0.102000,1,101100.0,782500.0,97.69,99.12,99.88,2.693,4.317,E,263.3,42.82,8.927,0.3462,0.1395,9.792,1.925,3.832,0.9082,514.0,255.1,1.548,2.084,112,17.0
4386,1266,45.23,9.67,2016-01-03,65,26.0,NaN,NaN,11.2,28.27,24.15,NaN,2.2690,2.0650,3.364,E,0.053580,1,100200.0,3285000.0,96.03,98.46,99.89,3.978,6.581,E,355.5,98.28,8.944,0.3722,0.1348,9.832,1.928,3.821,0.9067,514.5,255.1,1.548,2.084,112,17.0
4387,1266,45.23,9.67,2016-01-04,65,29.0,NaN,NaN,10.8,24.62,19.71,NaN,1.8980,1.3000,3.356,E,0.075220,1,99250.0,1760000.0,96.12,98.12,99.81,2.657,6.460,E,345.6,21.80,8.961,0.3969,0.1299,9.869,1.931,3.809,0.9054,515.0,255.0,1.548,2.083,112,17.0
4388,1266,45.23,9.67,2016-01-05,65,34.0,NaN,NaN,15.1,34.84,22.93,NaN,1.5550,1.0270,1.708,E,0.012900,1,99310.0,1429000.0,97.10,99.06,99.84,2.079,3.423,E,231.6,24.76,8.976,0.4204,0.1248,9.903,1.933,3.797,0.9042,515.5,254.9,1.548,2.083,112,17.0


In [240]:
support = mydf.drop(['IDStations', 'Altitude','Latitude', 'Longitude', 'Time', 'WE_mode_wind_direction_10m', 'WE_mode_wind_direction_100m'], axis=1)
support.head()

,AQ_pm10,AQ_pm25,AQ_co,AQ_nh3,AQ_nox,AQ_no2,AQ_so2,WE_temp_2m,WE_wind_speed_10m_mean,WE_wind_speed_10m_max,WE_tot_precipitation,WE_precipitation_t,WE_surface_pressure,WE_solar_radiation,WE_rh_min,WE_rh_mean,WE_rh_max,WE_wind_speed_100m_mean,WE_wind_speed_100m_max,WE_blh_layer_max,WE_blh_layer_min,EM_nh3_livestock_mm,EM_nh3_agr_soils,EM_nh3_agr_waste_burn,EM_nh3_sum,EM_nox_traffic,EM_nox_sum,EM_so2_sum,LI_pigs,LI_bovine,LA_hvi,LA_lvi,LA_land_use,LA_soil_use
4384,52.0,NaN,NaN,32.2,36.21,23.50,NaN,2.1610,0.9449,1.818,0.000009,0,101900.0,5330000.0,92.17,97.16,99.67,1.429,2.622,351.7,13.88,8.907,0.3187,0.1439,9.749,1.922,3.844,0.9099,513.5,255.2,1.548,2.084,112,17.0
4385,39.0,NaN,NaN,14.6,29.72,24.75,NaN,0.9301,1.4660,2.335,0.102000,1,101100.0,782500.0,97.69,99.12,99.88,2.693,4.317,263.3,42.82,8.927,0.3462,0.1395,9.792,1.925,3.832,0.9082,514.0,255.1,1.548,2.084,112,17.0
4386,26.0,NaN,NaN,11.2,28.27,24.15,NaN,2.2690,2.0650,3.364,0.053580,1,100200.0,3285000.0,96.03,98.46,99.89,3.978,6.581,355.5,98.28,8.944,0.3722,0.1348,9.832,1.928,3.821,0.9067,514.5,255.1,1.548,2.084,112,17.0
4387,29.0,NaN,NaN,10.8,24.62,19.71,NaN,1.8980,1.3000,3.356,0.075220,1,99250.0,1760000.0,96.12,98.12,99.81,2.657,6.460,345.6,21.80,8.961,0.3969,0.1299,9.869,1.931,3.809,0.9054,515.0,255.0,1.548,2.083,112,17.0
4388,34.0,NaN,NaN,15.1,34.84,22.93,NaN,1.5550,1.0270,1.708,0.012900,1,99310.0,1429000.0,97.10,99.06,99.84,2.079,3.423,231.6,24.76,8.976,0.4204,0.1248,9.903,1.933,3.797,0.9042,515.5,254.9,1.548,2.083,112,17.0


#Calculate parameters

In [241]:
# Calculate correlation
corr_matrix = support.corr()

# Calculate variance
variance = support.var()

# Calculate standard deviation
std_dev = support.std()

### Printing the results

In [242]:
# Print results
print("Correlation Matrix:\n", corr_matrix)


Correlation Matrix:
                           AQ_pm10  AQ_pm25  AQ_co    AQ_nh3    AQ_nox  \
AQ_pm10                  1.000000      NaN    NaN  0.526113  0.661738   
AQ_pm25                       NaN      NaN    NaN       NaN       NaN   
AQ_co                         NaN      NaN    NaN       NaN       NaN   
AQ_nh3                   0.526113      NaN    NaN  1.000000  0.276163   
AQ_nox                   0.661738      NaN    NaN  0.276163  1.000000   
AQ_no2                   0.690679      NaN    NaN  0.358236  0.922864   
AQ_so2                        NaN      NaN    NaN       NaN       NaN   
WE_temp_2m              -0.381498      NaN    NaN  0.164011 -0.617276   
WE_wind_speed_10m_mean  -0.339125      NaN    NaN -0.320985 -0.313539   
WE_wind_speed_10m_max   -0.355417      NaN    NaN -0.298297 -0.340069   
WE_tot_precipitation    -0.267920      NaN    NaN -0.266221 -0.197652   
WE_precipitation_t       0.015819      NaN    NaN -0.220748  0.010150   
WE_surface_pressure      0.455

In [243]:
print("Variance:\n", variance)


Variance:
 AQ_pm10                    3.645540e+02
AQ_pm25                             NaN
AQ_co                               NaN
AQ_nh3                     3.126433e+02
AQ_nox                     6.573766e+02
AQ_no2                     1.116944e+02
AQ_so2                              NaN
WE_temp_2m                 6.296560e+01
WE_wind_speed_10m_mean     5.717439e-01
WE_wind_speed_10m_max      1.285869e+00
WE_tot_precipitation       7.160602e-03
WE_precipitation_t         2.961169e-01
WE_surface_pressure        5.281998e+05
WE_solar_radiation         3.912745e+13
WE_rh_min                  3.451382e+01
WE_rh_mean                 1.631830e+01
WE_rh_max                  5.016558e+00
WE_wind_speed_100m_mean    1.791002e+00
WE_wind_speed_100m_max     3.661836e+00
WE_blh_layer_max           3.172470e+05
WE_blh_layer_min           2.112134e+03
EM_nh3_livestock_mm        6.240034e+01
EM_nh3_agr_soils           6.740130e+01
EM_nh3_agr_waste_burn      2.441259e-02
EM_nh3_sum                 1.

In [244]:
print("Standard Deviation:\n", std_dev)

Standard Deviation:
 AQ_pm10                    1.909330e+01
AQ_pm25                             NaN
AQ_co                               NaN
AQ_nh3                     1.768172e+01
AQ_nox                     2.563936e+01
AQ_no2                     1.056856e+01
AQ_so2                              NaN
WE_temp_2m                 7.935087e+00
WE_wind_speed_10m_mean     7.561375e-01
WE_wind_speed_10m_max      1.133961e+00
WE_tot_precipitation       8.462034e-02
WE_precipitation_t         5.441663e-01
WE_surface_pressure        7.267736e+02
WE_solar_radiation         6.255194e+06
WE_rh_min                  5.874846e+00
WE_rh_mean                 4.039592e+00
WE_rh_max                  2.239768e+00
WE_wind_speed_100m_mean    1.338283e+00
WE_wind_speed_100m_max     1.913593e+00
WE_blh_layer_max           5.632468e+02
WE_blh_layer_min           4.595796e+01
EM_nh3_livestock_mm        7.899389e+00
EM_nh3_agr_soils           8.209829e+00
EM_nh3_agr_waste_burn      1.562453e-01
EM_nh3_sum         

#Analisi della sensitività

In [245]:
support.head()

,AQ_pm10,AQ_pm25,AQ_co,AQ_nh3,AQ_nox,AQ_no2,AQ_so2,WE_temp_2m,WE_wind_speed_10m_mean,WE_wind_speed_10m_max,WE_tot_precipitation,WE_precipitation_t,WE_surface_pressure,WE_solar_radiation,WE_rh_min,WE_rh_mean,WE_rh_max,WE_wind_speed_100m_mean,WE_wind_speed_100m_max,WE_blh_layer_max,WE_blh_layer_min,EM_nh3_livestock_mm,EM_nh3_agr_soils,EM_nh3_agr_waste_burn,EM_nh3_sum,EM_nox_traffic,EM_nox_sum,EM_so2_sum,LI_pigs,LI_bovine,LA_hvi,LA_lvi,LA_land_use,LA_soil_use
4384,52.0,NaN,NaN,32.2,36.21,23.50,NaN,2.1610,0.9449,1.818,0.000009,0,101900.0,5330000.0,92.17,97.16,99.67,1.429,2.622,351.7,13.88,8.907,0.3187,0.1439,9.749,1.922,3.844,0.9099,513.5,255.2,1.548,2.084,112,17.0
4385,39.0,NaN,NaN,14.6,29.72,24.75,NaN,0.9301,1.4660,2.335,0.102000,1,101100.0,782500.0,97.69,99.12,99.88,2.693,4.317,263.3,42.82,8.927,0.3462,0.1395,9.792,1.925,3.832,0.9082,514.0,255.1,1.548,2.084,112,17.0
4386,26.0,NaN,NaN,11.2,28.27,24.15,NaN,2.2690,2.0650,3.364,0.053580,1,100200.0,3285000.0,96.03,98.46,99.89,3.978,6.581,355.5,98.28,8.944,0.3722,0.1348,9.832,1.928,3.821,0.9067,514.5,255.1,1.548,2.084,112,17.0
4387,29.0,NaN,NaN,10.8,24.62,19.71,NaN,1.8980,1.3000,3.356,0.075220,1,99250.0,1760000.0,96.12,98.12,99.81,2.657,6.460,345.6,21.80,8.961,0.3969,0.1299,9.869,1.931,3.809,0.9054,515.0,255.0,1.548,2.083,112,17.0
4388,34.0,NaN,NaN,15.1,34.84,22.93,NaN,1.5550,1.0270,1.708,0.012900,1,99310.0,1429000.0,97.10,99.06,99.84,2.079,3.423,231.6,24.76,8.976,0.4204,0.1248,9.903,1.933,3.797,0.9042,515.5,254.9,1.548,2.083,112,17.0


In [246]:
support.dtypes

AQ_pm10                    float64
AQ_pm25                    float64
AQ_co                      float64
AQ_nh3                     float64
AQ_nox                     float64
AQ_no2                     float64
AQ_so2                     float64
WE_temp_2m                 float64
WE_wind_speed_10m_mean     float64
WE_wind_speed_10m_max      float64
WE_tot_precipitation       float64
WE_precipitation_t           int64
WE_surface_pressure        float64
WE_solar_radiation         float64
WE_rh_min                  float64
WE_rh_mean                 float64
WE_rh_max                  float64
WE_wind_speed_100m_mean    float64
WE_wind_speed_100m_max     float64
WE_blh_layer_max           float64
WE_blh_layer_min           float64
EM_nh3_livestock_mm        float64
EM_nh3_agr_soils           float64
EM_nh3_agr_waste_burn      float64
EM_nh3_sum                 float64
EM_nox_traffic             float64
EM_nox_sum                 float64
EM_so2_sum                 float64
LI_pigs             

In [247]:
print(support.shape)
support

(1851, 34)


,AQ_pm10,AQ_pm25,AQ_co,AQ_nh3,AQ_nox,AQ_no2,AQ_so2,WE_temp_2m,WE_wind_speed_10m_mean,WE_wind_speed_10m_max,WE_tot_precipitation,WE_precipitation_t,WE_surface_pressure,WE_solar_radiation,WE_rh_min,WE_rh_mean,WE_rh_max,WE_wind_speed_100m_mean,WE_wind_speed_100m_max,WE_blh_layer_max,WE_blh_layer_min,EM_nh3_livestock_mm,EM_nh3_agr_soils,EM_nh3_agr_waste_burn,EM_nh3_sum,EM_nox_traffic,EM_nox_sum,EM_so2_sum,LI_pigs,LI_bovine,LA_hvi,LA_lvi,LA_land_use,LA_soil_use
4384,52.0,NaN,NaN,32.2,36.21,23.50,NaN,2.1610,0.9449,1.818,0.000009,0,101900.0,5330000.0,92.17,97.16,99.67,1.429,2.622,351.7,13.88,8.907,0.3187,0.1439,9.749,1.922,3.844,0.9099,513.5,255.2,1.548,2.084,112,17.0
4385,39.0,NaN,NaN,14.6,29.72,24.75,NaN,0.9301,1.4660,2.335,0.102000,1,101100.0,782500.0,97.69,99.12,99.88,2.693,4.317,263.3,42.82,8.927,0.3462,0.1395,9.792,1.925,3.832,0.9082,514.0,255.1,1.548,2.084,112,17.0
4386,26.0,NaN,NaN,11.2,28.27,24.15,NaN,2.2690,2.0650,3.364,0.053580,1,100200.0,3285000.0,96.03,98.46,99.89,3.978,6.581,355.5,98.28,8.944,0.3722,0.1348,9.832,1.928,3.821,0.9067,514.5,255.1,1.548,2.084,112,17.0
4387,29.0,NaN,NaN,10.8,24.62,19.71,NaN,1.8980,1.3000,3.356,0.075220,1,99250.0,1760000.0,96.12,98.12,99.81,2.657,6.460,345.6,21.80,8.961,0.3969,0.1299,9.869,1.931,3.809,0.9054,515.0,255.0,1.548,2.083,112,17.0
4388,34.0,NaN,NaN,15.1,34.84,22.93,NaN,1.5550,1.0270,1.708,0.012900,1,99310.0,1429000.0,97.10,99.06,99.84,2.079,3.423,231.6,24.76,8.976,0.4204,0.1248,9.903,1.933,3.797,0.9042,515.5,254.9,1.548,2.083,112,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571,60.0,NaN,NaN,18.5,99.34,32.95,NaN,4.4150,0.8309,1.184,0.030720,1,100200.0,2100000.0,98.05,99.49,99.98,1.516,2.506,290.9,28.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,601.4,250.7,1.548,2.085,112,NaN
6572,49.0,NaN,NaN,16.9,67.91,29.32,NaN,5.4910,1.2560,2.284,0.009877,0,100100.0,3031000.0,96.67,99.00,99.95,2.586,5.106,257.6,35.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,601.4,250.7,1.548,2.085,112,NaN
6573,32.0,NaN,NaN,11.2,59.09,29.58,NaN,5.1790,1.7810,2.940,0.000730,0,100300.0,3363000.0,94.67,98.14,99.79,4.160,6.820,223.8,25.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,601.4,250.7,1.548,2.084,112,NaN
6574,22.0,NaN,NaN,10.4,57.26,30.50,NaN,4.8350,0.8292,1.708,0.000075,0,101300.0,4521000.0,95.79,98.73,99.95,1.527,3.263,169.8,18.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,601.4,250.7,1.548,2.084,112,NaN


IN QUESTA CELLA DEVO RIMUOVERE I NaN ma non riesco a farlo

In [248]:
support = support.drop(['AQ_pm25', 'AQ_so2', 'AQ_co'], axis=1)
support.head()

,AQ_pm10,AQ_nh3,AQ_nox,AQ_no2,WE_temp_2m,WE_wind_speed_10m_mean,WE_wind_speed_10m_max,WE_tot_precipitation,WE_precipitation_t,WE_surface_pressure,WE_solar_radiation,WE_rh_min,WE_rh_mean,WE_rh_max,WE_wind_speed_100m_mean,WE_wind_speed_100m_max,WE_blh_layer_max,WE_blh_layer_min,EM_nh3_livestock_mm,EM_nh3_agr_soils,EM_nh3_agr_waste_burn,EM_nh3_sum,EM_nox_traffic,EM_nox_sum,EM_so2_sum,LI_pigs,LI_bovine,LA_hvi,LA_lvi,LA_land_use,LA_soil_use
4384,52.0,32.2,36.21,23.50,2.1610,0.9449,1.818,0.000009,0,101900.0,5330000.0,92.17,97.16,99.67,1.429,2.622,351.7,13.88,8.907,0.3187,0.1439,9.749,1.922,3.844,0.9099,513.5,255.2,1.548,2.084,112,17.0
4385,39.0,14.6,29.72,24.75,0.9301,1.4660,2.335,0.102000,1,101100.0,782500.0,97.69,99.12,99.88,2.693,4.317,263.3,42.82,8.927,0.3462,0.1395,9.792,1.925,3.832,0.9082,514.0,255.1,1.548,2.084,112,17.0
4386,26.0,11.2,28.27,24.15,2.2690,2.0650,3.364,0.053580,1,100200.0,3285000.0,96.03,98.46,99.89,3.978,6.581,355.5,98.28,8.944,0.3722,0.1348,9.832,1.928,3.821,0.9067,514.5,255.1,1.548,2.084,112,17.0
4387,29.0,10.8,24.62,19.71,1.8980,1.3000,3.356,0.075220,1,99250.0,1760000.0,96.12,98.12,99.81,2.657,6.460,345.6,21.80,8.961,0.3969,0.1299,9.869,1.931,3.809,0.9054,515.0,255.0,1.548,2.083,112,17.0
4388,34.0,15.1,34.84,22.93,1.5550,1.0270,1.708,0.012900,1,99310.0,1429000.0,97.10,99.06,99.84,2.079,3.423,231.6,24.76,8.976,0.4204,0.1248,9.903,1.933,3.797,0.9042,515.5,254.9,1.548,2.083,112,17.0


In [249]:
support = support.dropna()
support

,AQ_pm10,AQ_nh3,AQ_nox,AQ_no2,WE_temp_2m,WE_wind_speed_10m_mean,WE_wind_speed_10m_max,WE_tot_precipitation,WE_precipitation_t,WE_surface_pressure,WE_solar_radiation,WE_rh_min,WE_rh_mean,WE_rh_max,WE_wind_speed_100m_mean,WE_wind_speed_100m_max,WE_blh_layer_max,WE_blh_layer_min,EM_nh3_livestock_mm,EM_nh3_agr_soils,EM_nh3_agr_waste_burn,EM_nh3_sum,EM_nox_traffic,EM_nox_sum,EM_so2_sum,LI_pigs,LI_bovine,LA_hvi,LA_lvi,LA_land_use,LA_soil_use
4384,52.0,32.2,36.21,23.50,2.1610,0.9449,1.818,0.000009,0,101900.0,5330000.0,92.17,97.16,99.67,1.429,2.622,351.7,13.88,8.907,0.3187,0.1439,9.749,1.922,3.844,0.9099,513.5,255.2,1.548,2.084,112,17.0
4385,39.0,14.6,29.72,24.75,0.9301,1.4660,2.335,0.102000,1,101100.0,782500.0,97.69,99.12,99.88,2.693,4.317,263.3,42.82,8.927,0.3462,0.1395,9.792,1.925,3.832,0.9082,514.0,255.1,1.548,2.084,112,17.0
4386,26.0,11.2,28.27,24.15,2.2690,2.0650,3.364,0.053580,1,100200.0,3285000.0,96.03,98.46,99.89,3.978,6.581,355.5,98.28,8.944,0.3722,0.1348,9.832,1.928,3.821,0.9067,514.5,255.1,1.548,2.084,112,17.0
4387,29.0,10.8,24.62,19.71,1.8980,1.3000,3.356,0.075220,1,99250.0,1760000.0,96.12,98.12,99.81,2.657,6.460,345.6,21.80,8.961,0.3969,0.1299,9.869,1.931,3.809,0.9054,515.0,255.0,1.548,2.083,112,17.0
4388,34.0,15.1,34.84,22.93,1.5550,1.0270,1.708,0.012900,1,99310.0,1429000.0,97.10,99.06,99.84,2.079,3.423,231.6,24.76,8.976,0.4204,0.1248,9.903,1.933,3.797,0.9042,515.5,254.9,1.548,2.083,112,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5840,50.0,42.8,96.50,34.48,4.1010,0.8183,1.877,0.001527,0,101600.0,4225000.0,92.92,96.34,98.53,1.502,3.233,254.7,11.51,9.002,0.7423,0.1610,10.470,1.106,2.881,0.8955,521.4,253.9,1.548,2.085,112,17.0
5841,62.0,40.7,105.50,34.10,3.3690,1.4380,2.083,0.000138,0,102400.0,3343000.0,94.78,97.82,99.95,2.336,4.549,271.1,16.37,8.971,0.6313,0.1577,10.320,1.097,2.860,0.8926,521.4,253.9,1.548,2.085,112,17.0
5842,65.0,38.7,77.59,28.73,2.6440,1.0990,2.978,0.000275,1,102700.0,4482000.0,94.17,97.99,99.75,2.017,5.354,347.0,14.37,8.943,0.5322,0.1543,10.190,1.089,2.841,0.8899,521.4,253.9,1.548,2.084,112,17.0
5843,48.0,20.6,32.91,19.10,1.7840,1.3020,2.857,0.000097,0,102900.0,3991000.0,94.72,98.13,99.76,1.869,4.482,352.4,17.79,8.917,0.4461,0.1508,10.070,1.082,2.823,0.8875,521.4,253.9,1.548,2.084,112,17.0


## linear regression

In [250]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Separazione tra variabili indipendenti e variabile dipendente
X = support.drop('AQ_pm10', axis=1) # variabili indipendenti
y = support['AQ_pm10'] # variabile dipendente


# Addestramento del modello
model = LinearRegression().fit(X, y)

# Calcola la variazione percentuale di ciascuna variabile indipendente rispetto alla media
sens_vars = []
for col in X.columns:
    mean_val = np.mean(X[col])
    sens_val = mean_val * 0.1 # 10% di variazione rispetto alla media
    sens_vars.append((col, sens_val))

# Valuta l'effetto della variazione percentuale di ciascuna variabile indipendente sul valore della variabile dipendente
base_pred = model.predict(X) # valore della variabile dipendente con valori originali delle variabili indipendenti
sens_results = {}
for var, val in sens_vars:
    X_sens = X.copy()
    X_sens[var] = val
    pred_sens = model.predict(X_sens)
    sens_results[var] = ((pred_sens - base_pred) / base_pred) * 100

# Visualizza i risultati
print('Effetto percentuale delle variazioni delle variabili indipendenti sulla variabile dipendente:')
for var, val in sens_results.items():
    print(f'{var}: {val[0]:.2f}%')

Effetto percentuale delle variazioni delle variabili indipendenti sulla variabile dipendente:
AQ_nh3: -20.56%
AQ_nox: 0.61%
AQ_no2: -27.51%
WE_temp_2m: -0.73%
WE_wind_speed_10m_mean: -7.37%
WE_wind_speed_10m_max: -2.68%
WE_tot_precipitation: -0.23%
WE_precipitation_t: 0.17%
WE_surface_pressure: -945.24%
WE_solar_radiation: 0.91%
WE_rh_min: 27.37%
WE_rh_mean: -306.07%
WE_rh_max: 45.53%
WE_wind_speed_100m_mean: 6.18%
WE_wind_speed_100m_max: 2.97%
WE_blh_layer_max: 0.29%
WE_blh_layer_min: -0.12%
EM_nh3_livestock_mm: 18.43%
EM_nh3_agr_soils: -0.49%
EM_nh3_agr_waste_burn: 17.28%
EM_nh3_sum: -9.62%
EM_nox_traffic: 64.59%
EM_nox_sum: -135.44%
EM_so2_sum: 38.34%
LI_pigs: 52.66%
LI_bovine: 226.52%
LA_hvi: -167.14%
LA_lvi: 276.01%
LA_land_use: 0.00%
LA_soil_use: 0.01%


# Sensitivity of neural networks

In [251]:
X = mydf[['AQ_nh3', 'AQ_nox', 'AQ_no2', 'WE_temp_2m', 
          'WE_wind_speed_10m_mean', 'WE_wind_speed_10m_max', 'WE_tot_precipitation', 
          'WE_precipitation_t', 'WE_surface_pressure', 'WE_solar_radiation', 'WE_rh_min',
          'WE_rh_mean', 'WE_rh_max', 'WE_wind_speed_100m_mean', 'WE_wind_speed_100m_max',
          'WE_blh_layer_max', 'WE_blh_layer_min',	'EM_nh3_livestock_mm', 'EM_nh3_agr_soils',
          'EM_nh3_agr_waste_burn', 'EM_nh3_sum', 'EM_nox_traffic', 'EM_nox_sum', 'EM_so2_sum',
          'LI_pigs', 'LI_bovine',	'LA_hvi', 'LA_lvi',	'LA_land_use',	'LA_soil_use'
          ]]
y = mydf['AQ_pm10']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)

])
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')